# Homeward: AI-Powered missing persons finder

### ⚠️ !! Go to the linked GitHub project for the full demo !! ⚠️

This prototype showcases just an interactive demo of what the web application available on GitHub provides, please check the full project on the attached repository.
The command for the provisioning and deprovisioning of GCP resources are taken from the `setup.sh` and `destry.sh` of the main repository, so you will read `gcloud`, `bq` and `gsutil` commands there and there (instead of the python counterpart) for this reason.

### Lost in a sea of unstructured data

Every year, thousands of people go missing, and law enforcement agencies face a daunting challenge: **analyzing vast amounts of unstructured surveillance data** spread across multiple formats, locations, and timeframes. Traditional approaches require manual review of countless hours of video footage, photographs, and witness reports - a time-consuming process that can mean the difference between life and death.

### How BigQuery + Gemini can make the difference

**Homeward** demonstrates how BigQuery's AI capabilities can transform this critical real-world problem by:

🧠 **AI-Powered Content Analysis**: Using Google's Gemini multimodal models to automatically analyze surveillance footage and identify potential matches against missing persons reports and sightings.

📊 **Pattern Recognition at Scale**: Leveraging BigQuery's analytical power to find temporal and spatial patterns across data points.

🔍 **Semantic Search**: Enabling natural language queries against unstructured video content and image data, by leveraging embedding and Vector Search within BigQuery to search missing persons and sightings.

### What you'll see in this notebook

In this demo, we'll walk through:

1. **Setting up the GCP resources** - Creating datasets, external tables, and AI connections
2. **Missing persons case management** - Create and summarize (through Gemini) missing persons reports
4. **Sighting report management** - Create and summarize (through Gemini) sightings
4. **Semantic search** - Find missing persons reports starting from sightings and vice-versa through vector search in BigQuery.
3. **AI-Powered Video Analysis** - Using Gemini to analyze surveillance footage to find potential matches with missing persons reports
5. **Environment cleanup** - Destroy the GCP resources created for this demo
---



In [1]:
from datetime import date, time
from IPython.display import Video
import os
import subprocess
import secrets
import tempfile
import uuid

from google.cloud import bigquery

/Users/danilotrombino/Documents/Dev/Google/BQ Hackathon/missing-finder/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Setup GCP Project variables
PROJECT_ID = "hackaton-pre-submit"
LOCATION = "us-central1"
CONNECTION_ID = "homeward_gcp_connection"
DATASET_ID = "homeward"
BUCKET_NAME_PREFIX = "homeward_videos_"

In [3]:
# Utility functions
def run_command(cmd):
    """Run command and return success status"""
    try:
        result = subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
        print(f"✅ Success: {cmd}")
        if result.stdout.strip():
            print(f"   Output: {result.stdout.strip()}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed: {cmd}")
        if e.stderr:
            print(f"   Error: {e.stderr.strip()}")
        return False

def generate_bucket_name():
    """Generate a unique bucket name with random suffix"""
    random_suffix = secrets.token_hex(4)
    return f"{BUCKET_NAME_PREFIX}{random_suffix}"

In [11]:
# BQ Client
client = bigquery.Client(project=PROJECT_ID, location=LOCATION)

# 🔧 GCP Environment Setup
This section handles the initial setup of Google Cloud Platform resources required for the Homeward missing persons finder system. **Skip this section if you have already run the `setup.sh` available in the GitHub repository** 

✅ **What this section does:**
- Enablement of needed Google APIs
- Creation of BigQuery dataset
- Creation of Cloud Storage buckets for video recordings (used as source for the BQ's object table)
- Creation of BigQuery connections for AI/ML model integration
- IAM roles and permissions to BQ's service account
- External table that allows to analyze Google Cloud objects through BigQuery + Gemini


In [4]:
print(f"🚀 Setting up GCP Project: {PROJECT_ID}")
print(f"📍 Region: {LOCATION}")
print(f"🔗 Connection: {CONNECTION_ID}")
print(f"📊 Dataset: {DATASET_ID}")
print(f"🪣 Bucket name prefix: {BUCKET_NAME_PREFIX}")


# Execute setup commands
print("\n=== Setting Project Configuration ===")
run_command(f"gcloud config set project {PROJECT_ID}")

print("\n=== Verifying Project Access ===")
run_command(f"gcloud projects describe {PROJECT_ID}")

print("\n=== Enabling Required APIs ===")
required_apis = [
    "aiplatform.googleapis.com",
    "bigquery.googleapis.com", 
    "storage.googleapis.com"
]

for api in required_apis:
    run_command(f"gcloud services enable {api} --project={PROJECT_ID}")

print("\n=== Creating Storage Bucket ===")

# Generate new bucket name if needed
BUCKET_NAME = generate_bucket_name()
print(f"Generated new bucket name: {BUCKET_NAME}")

# Create the bucket
if run_command(f"gsutil mb -p {PROJECT_ID} -c STANDARD -l {LOCATION} gs://{BUCKET_NAME}"):
    print(f"✅ Created new bucket: gs://{BUCKET_NAME}")
    
    # Set bucket to private
    run_command(f"gsutil iam ch -d allUsers:objectViewer gs://{BUCKET_NAME}")
    run_command(f"gsutil iam ch -d allAuthenticatedUsers:objectViewer gs://{BUCKET_NAME}")
    print("✅ Bucket configured as private")


print("\n=== Creating BigQuery Connection ===")
run_command(f"bq mk --connection --location={LOCATION} --project_id={PROJECT_ID} --connection_type=CLOUD_RESOURCE {CONNECTION_ID} || true")

print("\n=== Creating BigQuery Dataset ===")  
run_command(f"bq mk --dataset --location={LOCATION} --project_id={PROJECT_ID} {DATASET_ID} || true")

print("\n=== Configuring IAM Permissions ===")
# Get BigQuery connection service account and grant bucket access
print("Getting BigQuery connection service account...")
connection_full_id = f"{PROJECT_ID}.{LOCATION}.{CONNECTION_ID}"
get_sa_cmd = f'bq show --format json --connection {connection_full_id} | python3 -c "import sys, json; data=json.load(sys.stdin); print(data[\'cloudResource\'][\'serviceAccountId\'])"'

try:
    result = subprocess.run(get_sa_cmd, shell=True, check=True, capture_output=True, text=True)
    service_account = result.stdout.strip()
    if service_account:
        print(f"Found service account: {service_account}")
        
        # Grant storage permissions
        run_command(f"gcloud storage buckets add-iam-policy-binding gs://{BUCKET_NAME} --member=serviceAccount:{service_account} --role=roles/storage.objectViewer")
        
        # Grant Vertex AI permissions  
        run_command(f"gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{service_account} --role=roles/aiplatform.user")
        
        print("✅ IAM permissions configured")
    else:
        print("❌ Could not retrieve service account")
except subprocess.CalledProcessError:
    print("❌ Failed to configure IAM permissions - may need manual setup")

print("\n✅ GCP Project setup completed!")
print(f"🔑 Project ID: {PROJECT_ID}")
print(f"📍 Region: {LOCATION}")
print(f"🪣 Storage Bucket: gs://{BUCKET_NAME}")
print(f"🔗 BigQuery Connection: {CONNECTION_ID}")
print(f"📊 Dataset: {DATASET_ID}")


🚀 Setting up GCP Project: hackaton-pre-submit
📍 Region: us-central1
🔗 Connection: homeward_gcp_connection
📊 Dataset: homeward
🪣 Bucket name prefix: homeward_videos_

=== Setting Project Configuration ===
✅ Success: gcloud config set project hackaton-pre-submit

=== Verifying Project Access ===
✅ Success: gcloud projects describe hackaton-pre-submit
   Output: createTime: '2025-09-22T20:36:19.917464Z'
lifecycleState: ACTIVE
name: hackaton-pre-submit
projectId: hackaton-pre-submit
projectNumber: '785373786347'

=== Enabling Required APIs ===
✅ Success: gcloud services enable aiplatform.googleapis.com --project=hackaton-pre-submit
✅ Success: gcloud services enable bigquery.googleapis.com --project=hackaton-pre-submit
✅ Success: gcloud services enable storage.googleapis.com --project=hackaton-pre-submit

=== Creating Storage Bucket ===
Generated new bucket name: homeward_videos_7fde5434
✅ Success: gsutil mb -p hackaton-pre-submit -c STANDARD -l us-central1 gs://homeward_videos_7fde5434
✅ C

# BigQuery DDLs Execution
Run this section to create the basic tables needed for the first part of the demo (Video analysis will be performed at the end of the notebook).
**Skip this section if you have already run the `setup.sh` available in the GitHub repository** 

✅ **What this section does:**
- Creation of missing persons table
- Creation of sightings table
- Creation of BQ ML model for embeddings

In [7]:
CREATE_MISSING_PERSONS_TABLE_DDL = f"""
CREATE TABLE IF NOT EXISTS `{DATASET_ID}.missing_persons` (
  /* Primary identifiers */
  id STRING NOT NULL OPTIONS(description="Unique case identifier"),
  case_number STRING OPTIONS(description="Official case reference number if available"),
  
  /* Personal Information */
  name STRING NOT NULL OPTIONS(description="First name of the missing person"),
  surname STRING NOT NULL OPTIONS(description="Last name of the missing person"),
  date_of_birth DATE NOT NULL OPTIONS(description="Date of birth of the missing person"),
  gender STRING NOT NULL OPTIONS(description="Gender (Male/Female/Other/Prefer not to say)"),
  
  /* Physical Description */
  height FLOAT64 OPTIONS(description="Height in centimeters"),
  weight FLOAT64 OPTIONS(description="Weight in kilograms"),
  hair_color STRING OPTIONS(description="Hair color"),
  eye_color STRING OPTIONS(description="Eye color"),
  distinguishing_marks STRING OPTIONS(description="Scars, tattoos, birthmarks, unique features"),
  clothing_description STRING OPTIONS(description="Description of clothing and accessories when last seen"),
  
  /* Last Seen Information */
  last_seen_date DATE NOT NULL OPTIONS(description="Date when person was last seen"),
  last_seen_time TIME OPTIONS(description="Time when person was last seen"),
  last_seen_address STRING NOT NULL OPTIONS(description="Street address where person was last seen"),
  last_seen_city STRING NOT NULL OPTIONS(description="City where person was last seen"),
  last_seen_country STRING NOT NULL OPTIONS(description="Country where person was last seen"),
  last_seen_postal_code STRING OPTIONS(description="Postal code where person was last seen"),
  last_seen_latitude FLOAT64 OPTIONS(description="Latitude coordinates of last seen location"),
  last_seen_longitude FLOAT64 OPTIONS(description="Longitude coordinates of last seen location"),
  last_seen_geo GEOGRAPHY OPTIONS(description="SFS position of last seen location"),

  /* Case Details */
  circumstances STRING NOT NULL OPTIONS(description="Detailed description of circumstances of disappearance"),
  priority STRING NOT NULL OPTIONS(description="Priority level (High/Medium/Low)"),
  status STRING NOT NULL OPTIONS(description="Case status (Active/Resolved/Suspended)"),
  description STRING OPTIONS(description="General case description"),
  
  /* Additional Information */
  medical_conditions STRING OPTIONS(description="Medical conditions or mental health information"),
  additional_info STRING OPTIONS(description="Any other relevant information"),
  
  /* Media */
  photo_url STRING OPTIONS(description="URL to photo of missing person"),
  
  /* Contact Information (Reporter) */
  reporter_name STRING NOT NULL OPTIONS(description="Name of person reporting the missing person"),
  reporter_phone STRING NOT NULL OPTIONS(description="Phone number of reporter"),
  reporter_email STRING OPTIONS(description="Email address of reporter"),
  relationship STRING NOT NULL OPTIONS(description="Relationship of reporter to missing person"),
  
  /* Metadata */
  created_date TIMESTAMP NOT NULL OPTIONS(description="Date and time when case was created"),
  updated_date TIMESTAMP NOT NULL OPTIONS(description="Date and time when case was last updated"),
  
  /* AI-Generated Content */
  ml_summary STRING OPTIONS(description="AI-generated comprehensive summary of the missing person case for analysis and matching"),
  ml_summary_embedding ARRAY<FLOAT64> OPTIONS(description="Embedding vector of the AI-generated summary for similarity search and matching")

  --TODO Add Missing Photo Embedding
)
PARTITION BY DATE(created_date)
CLUSTER BY status, priority, last_seen_city
OPTIONS(
  description="Table storing comprehensive missing person case information for the Homeward application",
  labels=[("environment", "hackathon"), ("application", "{DATASET_ID}"), ("data_type", "missing_persons")]
);
"""

In [8]:
CREATE_SIGHTINGS_TABLE_DDL = f"""
CREATE TABLE IF NOT EXISTS `{DATASET_ID}.sightings` (
  /* Primary identifiers */
  id STRING NOT NULL OPTIONS(description="Unique sighting identifier"),
  sighting_number STRING OPTIONS(description="Official sighting reference number if available"),
  
  /* Sighting Information */
  sighted_date DATE NOT NULL OPTIONS(description="Date when person was sighted"),
  sighted_time TIME OPTIONS(description="Time when person was sighted"),
  sighted_address STRING NOT NULL OPTIONS(description="Street address where person was sighted"),
  sighted_city STRING NOT NULL OPTIONS(description="City where person was sighted"),
  sighted_country STRING NOT NULL OPTIONS(description="Country where person was sighted"),
  sighted_postal_code STRING OPTIONS(description="Postal code where person was sighted"),
  sighted_latitude FLOAT64 OPTIONS(description="Latitude coordinates of sighting location"),
  sighted_longitude FLOAT64 OPTIONS(description="Longitude coordinates of sighting location"),
  sighted_geo GEOGRAPHY OPTIONS(description="SFS position of sighting location"),
  
  /* Person Description */
  apparent_gender STRING OPTIONS(description="Apparent gender of sighted person"),
  apparent_age_range STRING OPTIONS(description="Estimated age range (e.g., '20-30', '40-50')"),
  height_estimate FLOAT64 OPTIONS(description="Estimated height in centimeters"),
  weight_estimate FLOAT64 OPTIONS(description="Estimated weight in kilograms"),
  hair_color STRING OPTIONS(description="Observed hair color"),
  eye_color STRING OPTIONS(description="Observed eye color"),
  clothing_description STRING OPTIONS(description="Description of clothing and accessories observed"),
  distinguishing_features STRING OPTIONS(description="Notable features, marks, or characteristics observed"),
  
  /* Sighting Details */
  description STRING NOT NULL OPTIONS(description="Detailed description of the sighting"),
  circumstances STRING OPTIONS(description="Circumstances under which person was sighted"),
  confidence_level STRING NOT NULL OPTIONS(description="Reporter's confidence level (High/Medium/Low)"),
  photo_url STRING OPTIONS(description="URL to photo of sighted person if available"),
  video_url STRING OPTIONS(description="URL to video footage if available"),
  
  /* Source Information */
  source_type STRING NOT NULL OPTIONS(description="Source of sighting (Witness/Manual_Entry/Other)"),
  witness_name STRING OPTIONS(description="Name of witness (if applicable)"),
  witness_phone STRING OPTIONS(description="Phone number of witness (if applicable)"),
  witness_email STRING OPTIONS(description="Email address of witness (if applicable)"),
  video_analytics_result_id STRING OPTIONS(description="Reference to video_analytics_results.id if converted from AI detection"),
  
  /* Status and Processing */
  status STRING NOT NULL OPTIONS(description="Sighting status (New/Under_Review/Verified/False_Positive/Archived)"),
  priority STRING NOT NULL OPTIONS(description="Priority level (High/Medium/Low)"),
  verified BOOLEAN NOT NULL OPTIONS(description="Whether sighting has been verified"),
  
  /* Metadata */
  created_date TIMESTAMP NOT NULL OPTIONS(description="Date and time when sighting was created"),
  updated_date TIMESTAMP NOT NULL OPTIONS(description="Date and time when sighting was last updated"),
  created_by STRING OPTIONS(description="User or system that created the sighting"),
  notes STRING OPTIONS(description="Additional notes or comments about the sighting"),
  
  /* AI-Generated Content */
  ml_summary STRING OPTIONS(description="AI-generated comprehensive summary of the sighting for analysis and matching"),
  ml_summary_embedding ARRAY<FLOAT64> OPTIONS(description="Embedding vector of the AI-generated summary for similarity search and matching")
)
PARTITION BY DATE(created_date)
CLUSTER BY status, priority, sighted_city, source_type
OPTIONS(
  description="Table storing sighting reports that can be linked to missing person cases",
  labels=[("environment", "hackathon"), ("application", "{DATASET_ID}"), ("data_type", "sightings")]
);
"""

In [9]:
CREATE_EMBEDDING_MODEL_DDL = f"""
CREATE OR REPLACE MODEL `{DATASET_ID}.text_embedding_model`
REMOTE WITH CONNECTION `{PROJECT_ID}.{LOCATION}.{CONNECTION_ID}`
OPTIONS (
  endpoint = 'text-embedding-004'
);
"""

In [12]:
print("🔧 Executing CREATE_MISSING_PERSONS_TABLE_DDL...")
job_config = bigquery.QueryJobConfig()
query_job = client.query(CREATE_MISSING_PERSONS_TABLE_DDL, job_config=job_config)
results = query_job.result()
print("✅ CREATE_MISSING_PERSONS_TABLE_DDL executed successfully!")
print(f"🔧 Query job completed: {query_job.job_id}")

🔧 Executing CREATE_MISSING_PERSONS_TABLE_DDL...
✅ CREATE_MISSING_PERSONS_TABLE_DDL executed successfully!
🔧 Query job completed: 33ed6dd6-1803-4fdf-9244-2bd9b701e951


In [13]:
print("🔧 Executing CREATE_SIGHTINGS_TABLE_DDL...")
job_config = bigquery.QueryJobConfig()
query_job = client.query(CREATE_SIGHTINGS_TABLE_DDL, job_config=job_config)
results = query_job.result()
print("✅ CREATE_SIGHTINGS_TABLE_DDL executed successfully!")
print(f"🔧 Query job completed: {query_job.job_id}")

🔧 Executing CREATE_SIGHTINGS_TABLE_DDL...
✅ CREATE_SIGHTINGS_TABLE_DDL executed successfully!
🔧 Query job completed: 6320044b-ace3-4b87-b6f6-555de7fbce19


In [14]:
print("🔧 Executing CREATE_EMBEDDING_MODEL_DDL...")
job_config = bigquery.QueryJobConfig()
query_job = client.query(CREATE_EMBEDDING_MODEL_DDL, job_config=job_config)
# In case of error retry after some seconds, there could be a delay in IAM permissions propagation
results = query_job.result()
print("✅ CREATE_EMBEDDING_MODEL_DDL executed successfully!")
print(f"🔧 Query job completed: {query_job.job_id}")

🔧 Executing CREATE_EMBEDDING_MODEL_DDL...
✅ CREATE_EMBEDDING_MODEL_DDL executed successfully!
🔧 Query job completed: b09e5837-936b-4859-a245-e3f471f0006f


# 🎬 Let's start with the demo
---

# 👤 Missing Person Case Management

This section demonstrates how to register and manage missing person cases in.
Each case includes information that can help law enforcement agencies track and coordinate search efforts.
These includes:
- Name and surname
- Date of birth
- Gender
- Approximate Height
- Approximate weight
- Hair color
- Eye color
- Distinguishing marks
- Clothing at the moment of the missing
- Coordinates of last place the person has been seen
- Medical conditions
- Photo
- Reporter information


🎯 **Key capabilities:**
- Create missing person profiles
- Provide geo-points data types to enable BigQuery geo-queries
- Store case summary in BigQuery through **Gemini integration**

📋 **Use case:** 
Law enforcement agencies receive a missing person report and need to register it in the system for coordinated search efforts across multiple surveillance networks. Such reports are processed and summarized to enable NL-queries and semantic searches

In [15]:
# Sample data for testing - this will be passed as parameters to the BQ Query. Note, the image comes from my sessioniza page :)
sample_missing_person = {
    "id": str(uuid.uuid4()),
    "case_number": "CASE-2025-0901-0001",
    "name": "John",
    "surname": "Doe",
    "date_of_birth": date(1990, 1, 15),
    "gender": "Male",
    "height": 175.0,
    "weight": 70.0,
    "hair_color": "Brown",
    "eye_color": "Green",
    "distinguishing_marks": "Tattoo on right arm",
    "clothing_description": "Black jacket, blue jeans, white sneakers",
    "last_seen_date": date(2025, 9, 1),
    "last_seen_time": time(18, 45, 0),
    "last_seen_address": "456 Oak Avenue",
    "last_seen_city": "San Francisco",
    "last_seen_country": "USA",
    "last_seen_postal_code": "94103",
    "last_seen_latitude": 37.7849,
    "last_seen_longitude": -122.4094,
    "circumstances": "Failed to return home after evening jog",
    "priority": "High",
    "status": "Active",
    "description": "Missing marathon runner",
    "medical_conditions": "Diabetes - requires medication",
    "additional_info": "Regular jogger, knows the area well",
    "photo_url": "https://sessionize.com/image/1447-400o400o2-NWmSNCUaND99mbwapa6z6.jpg",
    "reporter_name": "Jane Doe",
    "reporter_phone": "4155559876",
    "reporter_email": "jane.doe@email.com",
    "relationship": "Wife",
}

print(f"📋 Sample case ID: {sample_missing_person['id']}")
print(f"📋 Sample case number: {sample_missing_person['case_number']}")
print(
    f"👤 Sample person: {sample_missing_person['name']} {sample_missing_person['surname']}"
)

📋 Sample case ID: e0ffda51-a883-44ee-9c20-5cbdb8064f2e
📋 Sample case number: CASE-2025-0901-0001
👤 Sample person: John Doe


In [18]:
# Little hack to generate LLM summary on insert leveraging MERGE
MISSING_PERSON_INSERT_QUERY = f"""
MERGE `{DATASET_ID}.missing_persons` AS target
USING (
  SELECT
    @id AS id,
    @case_number AS case_number,
    @name AS name,
    @surname AS surname,
    @date_of_birth AS date_of_birth,
    @gender AS gender,
    @height AS height,
    @weight AS weight,
    @hair_color AS hair_color,
    @eye_color AS eye_color,
    @distinguishing_marks AS distinguishing_marks,
    @clothing_description AS clothing_description,
    @last_seen_date AS last_seen_date,
    @last_seen_time AS last_seen_time,
    @last_seen_address AS last_seen_address,
    @last_seen_city AS last_seen_city,
    @last_seen_country AS last_seen_country,
    @last_seen_postal_code AS last_seen_postal_code,
    @last_seen_latitude AS last_seen_latitude,
    @last_seen_longitude AS last_seen_longitude,
    CASE
      WHEN @last_seen_latitude IS NOT NULL AND @last_seen_longitude IS NOT NULL
      THEN ST_GEOGPOINT(@last_seen_longitude, @last_seen_latitude)
      ELSE NULL
    END AS last_seen_geo,
    @circumstances AS circumstances,
    @priority AS priority,
    @status AS status,
    @description AS description,
    @medical_conditions AS medical_conditions,
    @additional_info AS additional_info,
    @photo_url AS photo_url,
    @reporter_name AS reporter_name,
    @reporter_phone AS reporter_phone,
    @reporter_email AS reporter_email,
    @relationship AS relationship,
    CURRENT_TIMESTAMP() AS created_date,
    CURRENT_TIMESTAMP() AS updated_date,
    AI.GENERATE(
      CONCAT(
        'Generate a comprehensive summary paragraph for this missing person case for law enforcement analysis and matching purposes. ',
        'Write it as a single, flowing, discursive paragraph without bullet points, lists, or structured formatting. ',
        'Include key identifying features, circumstances, and critical search information in narrative form. ',
        'Return only the summary paragraph without any introduction, conclusion, or additional commentary from the model. ',
        'Person: ', @name, ' ', @surname, ', ',
        'Age: ', CAST(DATE_DIFF(CURRENT_DATE(), @date_of_birth, YEAR) AS STRING), ' years old, ',
        'Gender: ', @gender, ', ',
        CASE
          WHEN @height IS NOT NULL THEN CONCAT('Height: ', CAST(@height AS STRING), 'cm, ')
          ELSE ''
        END,
        CASE
          WHEN @weight IS NOT NULL THEN CONCAT('Weight: ', CAST(@weight AS STRING), 'kg, ')
          ELSE ''
        END,
        CASE
          WHEN @hair_color IS NOT NULL THEN CONCAT('Hair: ', @hair_color, ', ')
          ELSE ''
        END,
        CASE
          WHEN @eye_color IS NOT NULL THEN CONCAT('Eyes: ', @eye_color, ', ')
          ELSE ''
        END,
        CASE
          WHEN @distinguishing_marks IS NOT NULL THEN CONCAT('Distinguishing marks: ', @distinguishing_marks, '. ')
          ELSE ''
        END,
        CASE
          WHEN @clothing_description IS NOT NULL THEN CONCAT('Last seen wearing: ', @clothing_description, '. ')
          ELSE ''
        END,
        'Last seen on ', CAST(@last_seen_date AS STRING),
        CASE
          WHEN @last_seen_time IS NOT NULL THEN CONCAT(' at ', CAST(@last_seen_time AS STRING))
          ELSE ''
        END,
        ' in ', @last_seen_city, ', ', @last_seen_country, '. ',
        'Location: ', @last_seen_address,
        CASE
          WHEN @last_seen_postal_code IS NOT NULL THEN CONCAT(', ', @last_seen_postal_code)
          ELSE ''
        END,
        '. Circumstances: ', @circumstances, '. ',
        CASE
          WHEN @medical_conditions IS NOT NULL THEN CONCAT('Medical conditions: ', @medical_conditions, '. ')
          ELSE ''
        END,
        CASE
          WHEN @additional_info IS NOT NULL THEN CONCAT('Additional information: ', @additional_info, '. ')
          ELSE ''
        END
      ),
      connection_id => '{PROJECT_ID}.{LOCATION}.{CONNECTION_ID}',
      endpoint => 'gemini-2.5-flash',
      model_params => JSON '{{"generation_config": {{"temperature": 0}}}}'
    ).result AS ml_summary
) AS source
ON target.id = source.id
WHEN NOT MATCHED THEN
  INSERT (
    id, case_number, name, surname, date_of_birth, gender,
    height, weight, hair_color, eye_color, distinguishing_marks, clothing_description,
    last_seen_date, last_seen_time, last_seen_address, last_seen_city, last_seen_country,
    last_seen_postal_code, last_seen_latitude, last_seen_longitude, last_seen_geo,
    circumstances, priority, status, description, medical_conditions, additional_info,
    photo_url, reporter_name, reporter_phone, reporter_email, relationship,
    created_date, updated_date, ml_summary
  )
  VALUES (
    source.id, source.case_number, source.name, source.surname, source.date_of_birth, source.gender,
    source.height, source.weight, source.hair_color, source.eye_color, source.distinguishing_marks, source.clothing_description,
    source.last_seen_date, source.last_seen_time, source.last_seen_address, source.last_seen_city, source.last_seen_country,
    source.last_seen_postal_code, source.last_seen_latitude, source.last_seen_longitude, source.last_seen_geo,
    source.circumstances, source.priority, source.status, source.description, source.medical_conditions, source.additional_info,
    source.photo_url, source.reporter_name, source.reporter_phone, source.reporter_email, source.relationship,
    source.created_date, source.updated_date, source.ml_summary
  );
"""

# Query to verify the record
VERIFY_RECORD_QUERY = f"""
SELECT
    id,
    case_number,
    name,
    surname,
    date_of_birth,
    gender,
    height,
    weight,
    hair_color,
    eye_color,
    distinguishing_marks,
    clothing_description,
    last_seen_date,
    last_seen_time,
    last_seen_city,
    circumstances,
    priority,
    status,
    medical_conditions,
    additional_info,
    reporter_name,
    relationship,
    created_date,
    ml_summary
FROM `{DATASET_ID}.missing_persons`
WHERE id = @case_id
ORDER BY created_date DESC
LIMIT 1;
"""

In [19]:
# Execute the parameterized missing person insert query with ML summary generation

try:
    print("Executing parameterized missing person insert with ML summary generation...")

    # Configure the query job with parameters
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("id", "STRING", sample_missing_person["id"]),
            bigquery.ScalarQueryParameter(
                "case_number", "STRING", sample_missing_person["case_number"]
            ),
            bigquery.ScalarQueryParameter(
                "name", "STRING", sample_missing_person["name"]
            ),
            bigquery.ScalarQueryParameter(
                "surname", "STRING", sample_missing_person["surname"]
            ),
            bigquery.ScalarQueryParameter(
                "date_of_birth", "DATE", sample_missing_person["date_of_birth"]
            ),
            bigquery.ScalarQueryParameter(
                "gender", "STRING", sample_missing_person["gender"]
            ),
            bigquery.ScalarQueryParameter(
                "height", "FLOAT64", sample_missing_person["height"]
            ),
            bigquery.ScalarQueryParameter(
                "weight", "FLOAT64", sample_missing_person["weight"]
            ),
            bigquery.ScalarQueryParameter(
                "hair_color", "STRING", sample_missing_person["hair_color"]
            ),
            bigquery.ScalarQueryParameter(
                "eye_color", "STRING", sample_missing_person["eye_color"]
            ),
            bigquery.ScalarQueryParameter(
                "distinguishing_marks",
                "STRING",
                sample_missing_person["distinguishing_marks"],
            ),
            bigquery.ScalarQueryParameter(
                "clothing_description",
                "STRING",
                sample_missing_person["clothing_description"],
            ),
            bigquery.ScalarQueryParameter(
                "last_seen_date", "DATE", sample_missing_person["last_seen_date"]
            ),
            bigquery.ScalarQueryParameter(
                "last_seen_time", "TIME", sample_missing_person["last_seen_time"]
            ),
            bigquery.ScalarQueryParameter(
                "last_seen_address",
                "STRING",
                sample_missing_person["last_seen_address"],
            ),
            bigquery.ScalarQueryParameter(
                "last_seen_city", "STRING", sample_missing_person["last_seen_city"]
            ),
            bigquery.ScalarQueryParameter(
                "last_seen_country",
                "STRING",
                sample_missing_person["last_seen_country"],
            ),
            bigquery.ScalarQueryParameter(
                "last_seen_postal_code",
                "STRING",
                sample_missing_person["last_seen_postal_code"],
            ),
            bigquery.ScalarQueryParameter(
                "last_seen_latitude",
                "FLOAT64",
                sample_missing_person["last_seen_latitude"],
            ),
            bigquery.ScalarQueryParameter(
                "last_seen_longitude",
                "FLOAT64",
                sample_missing_person["last_seen_longitude"],
            ),
            bigquery.ScalarQueryParameter(
                "circumstances", "STRING", sample_missing_person["circumstances"]
            ),
            bigquery.ScalarQueryParameter(
                "priority", "STRING", sample_missing_person["priority"]
            ),
            bigquery.ScalarQueryParameter(
                "status", "STRING", sample_missing_person["status"]
            ),
            bigquery.ScalarQueryParameter(
                "description", "STRING", sample_missing_person["description"]
            ),
            bigquery.ScalarQueryParameter(
                "medical_conditions",
                "STRING",
                sample_missing_person["medical_conditions"],
            ),
            bigquery.ScalarQueryParameter(
                "additional_info", "STRING", sample_missing_person["additional_info"]
            ),
            bigquery.ScalarQueryParameter(
                "photo_url", "STRING", sample_missing_person["photo_url"]
            ),
            bigquery.ScalarQueryParameter(
                "reporter_name", "STRING", sample_missing_person["reporter_name"]
            ),
            bigquery.ScalarQueryParameter(
                "reporter_phone", "STRING", sample_missing_person["reporter_name"]
            ),
            bigquery.ScalarQueryParameter(
                "reporter_email", "STRING", sample_missing_person["reporter_email"]
            ),
            bigquery.ScalarQueryParameter(
                "relationship", "STRING", sample_missing_person["relationship"]
            ),
        ]
    )

    # Execute the parameterized query
    query_job = client.query(MISSING_PERSON_INSERT_QUERY, job_config=job_config)
    results = query_job.result()  # Wait for the query to complete

    print("✅ Missing person record inserted successfully, including AI-generated summary!")
    print(f"🔑 Case ID: {sample_missing_person['id']}")
    print(f"📋 Case Number: {sample_missing_person['case_number']}")
    print(
        f"👤 Person: {sample_missing_person['name']} {sample_missing_person['surname']}"
    )
    print(f"🔧 Query job completed: {query_job.job_id}")

except Exception as e:
    print(f"❌ Error executing parameterized insert query: {str(e)}")
    print(f"Error type: {type(e).__name__}")

    # Print more detailed error information if available
    if hasattr(e, "errors") and e.errors:
        for error in e.errors:
            print(f"Error details: {error}")
    if hasattr(e, "message"):
        print(f"Error message: {e.message}")

Executing parameterized missing person insert with ML summary generation...
✅ Missing person record inserted successfully, including AI-generated summary!
🔑 Case ID: e0ffda51-a883-44ee-9c20-5cbdb8064f2e
📋 Case Number: CASE-2025-0901-0001
👤 Person: John Doe
🔧 Query job completed: 085b085f-6e33-4aa0-8f7c-5b578c40135e


In [20]:
try:
    print("Querying the inserted record to view AI-generated summary...")
    job_config = bigquery.QueryJobConfig(
        query_parameters=[bigquery.ScalarQueryParameter("case_id", "STRING", sample_missing_person["id"])]
    )

    # Execute the verification query
    query_job = client.query(VERIFY_RECORD_QUERY, job_config=job_config)
    results = query_job.result()

    print("✅ Verification query executed successfully!")
    print(f"Total rows returned: {results.total_rows}")

    if results.total_rows > 0:
        print("\n" + "=" * 80)
        print("INSERTED MISSING PERSON RECORD WITH AI-GENERATED SUMMARY")
        print("=" * 80)

        for row in results:
            print(f"📋 Case ID: {row.id}")
            print(f"📋 Case Number: {row.case_number}")
            print(f"👤 Name: {row.name} {row.surname}")
            print(f"🎂 Date of Birth: {row.date_of_birth}")
            print(f"⚧ Gender: {row.gender}")
            print(f"📏 Physical: {row.height}cm, {row.weight}kg")
            print(f"👁️ Features: {row.hair_color} hair, {row.eye_color} eyes")
            print(f"🔍 Distinguishing Marks: {row.distinguishing_marks}")
            print(f"👕 Clothing: {row.clothing_description}")
            print(
                f"📍 Last Seen: {row.last_seen_date} at {row.last_seen_time} in {row.last_seen_city}"
            )
            print(f"⚠️ Circumstances: {row.circumstances}")
            print(f"🏥 Medical Conditions: {row.medical_conditions}")
            print(f"ℹ️ Additional Info: {row.additional_info}")
            print(f"📞 Reporter: {row.reporter_name} ({row.relationship})")
            print(f"🔴 Priority: {row.priority}")
            print(f"📊 Status: {row.status}")
            print(f"📅 Created: {row.created_date}")

            print("\n" + "-" * 80)
            print("🤖 AI-GENERATED SUMMARY:")
            print("-" * 80)
            print(f"{row.ml_summary}")
            print("-" * 80)

            # Calculate summary statistics
            summary_length = len(row.ml_summary) if row.ml_summary else 0
            word_count = len(row.ml_summary.split()) if row.ml_summary else 0
            print("\n📊 Summary Statistics:")
            print(f"   • Length: {summary_length} characters")
            print(f"   • Word count: {word_count} words")
            print(
                f"   • Format: {'Discursive paragraph' if summary_length > 100 else 'Short summary'}"
            )

    else:
        print(f"⚠️ No records found with case ID '{sample_missing_person['id']}'")

except Exception as e:
    print(f"❌ Error querying inserted record: {str(e)}")
    print(f"Error type: {type(e).__name__}")

    # Print more detailed error information if available
    if hasattr(e, "errors") and e.errors:
        for error in e.errors:
            print(f"Error details: {error}")

Querying the inserted record to view AI-generated summary...
✅ Verification query executed successfully!
Total rows returned: 1

INSERTED MISSING PERSON RECORD WITH AI-GENERATED SUMMARY
📋 Case ID: e0ffda51-a883-44ee-9c20-5cbdb8064f2e
📋 Case Number: CASE-2025-0901-0001
👤 Name: John Doe
🎂 Date of Birth: 1990-01-15
⚧ Gender: Male
📏 Physical: 175.0cm, 70.0kg
👁️ Features: Brown hair, Green eyes
🔍 Distinguishing Marks: Tattoo on right arm
👕 Clothing: Black jacket, blue jeans, white sneakers
📍 Last Seen: 2025-09-01 at 18:45:00 in San Francisco
⚠️ Circumstances: Failed to return home after evening jog
🏥 Medical Conditions: Diabetes - requires medication
ℹ️ Additional Info: Regular jogger, knows the area well
📞 Reporter: Jane Doe (Wife)
🔴 Priority: High
📊 Status: Active
📅 Created: 2025-09-22 22:45:23.785951+00:00

--------------------------------------------------------------------------------
🤖 AI-GENERATED SUMMARY:
------------------------------------------------------------------------------

# 👁️ Sighting Reports Management

This section handles the registration and management of potential sightings related to missing person cases. Sightings can come from various sources including manual reports or witness accounts.
Sighting information includes:
- Sighting description
- Sighting date
- Sighted time
- Sighted alocation
- Apparent gender
- Age range
- Estimate height
- Distinguish features
- Clothing description
- Witness information

🔍 **Core functionality:**
- Register sightings with location and timestamp data
- Store case summary in BigQuery through **Gemini integration**

💡 **Use Case:** When a witness reports seeing someone matching a missing person's description, or when AI analysis identifies a potential match in surveillance footage, this system captures and organizes that critical information for investigators."

In [21]:
# Sample sighting data that perfectly matches our missing person John Doe
sample_sighting = {
    "id": str(uuid.uuid4()),
    "sighting_number": "SIGHT-2024-070",
    "description": "30 to 35-year-old male jogger, tall, near 70kg, brown hair, green eyes, wearing black jacket, blue jeans, white sneakers, has distinctive tattoo on right arm, appeared disoriented, matches missing diabetic runner from San Francisco",
    "sighted_date": date(2024, 8, 21),  # Day after John went missing
    "sighted_time": time(8, 30, 0),
    "sighted_address": "Golden Gate Park, near the jogging trail",
    "sighted_city": "San Francisco",
    "sighted_country": "USA",
    "sighted_postal_code": "94117",
    "sighted_latitude": 37.7694,
    "sighted_longitude": -122.4862,
    "apparent_gender": "Male",
    "apparent_age_range": "30-40",
    "height_estimate": 175.0,
    "hair_color": "Brown",
    "clothing_description": "Black jacket, blue jeans, white sneakers",
    "distinguishing_features": "Tattoo on right arm",
    "source_type": "Witness",
    "witness_name": "Maria Rodriguez",
    "witness_phone": "4155551234",
    "witness_email": "maria.rodriguez@email.com",
    "circumstances": "Male marathon runner sitting on bench looking disoriented and in medical distress, appeared to be diabetic episode, matches description of missing person John Doe who requires diabetes medication, was wearing identical clothing to missing person report",
    "confidence_level": "High",
    "photo_url": "https://example.com/sightings/sight_2024_078.jpg",
    "status": "New",
    "priority": "High",
    "verified": False,
    "notes": "Witness is a regular jogger in the area and specifically noted the person matched the missing person flyers, person appeared to be having medical emergency consistent with diabetes, exact clothing match including distinctive tattoo on right arm",
}

print(f"📋 Sighting ID: {sample_sighting['id']}")
print(f"📋 Sighting Number: {sample_sighting['sighting_number']}")
print(f"👁️ Person Description: {sample_sighting['description'][:100]}...")
print(
    f"📍 Location: {sample_sighting['sighted_address']}, {sample_sighting['sighted_city']}"
)
print(
    f"📅 Sighting Date: {sample_sighting['sighted_date']} at {sample_sighting['sighted_time']}"
)
print(f"📞 Witness: {sample_sighting['witness_name']}")
print(f"⭐ Confidence Level: {sample_sighting['confidence_level']}")
print(f"🚨 Source Type: {sample_sighting['source_type']}")
print(f"🔴 Priority: {sample_sighting['priority']}")
print(f"✅ Verified: {sample_sighting['verified']}")

📋 Sighting ID: 5945a5b3-9063-46b8-b5e8-63213571a028
📋 Sighting Number: SIGHT-2024-070
👁️ Person Description: 30 to 35-year-old male jogger, tall, near 70kg, brown hair, green eyes, wearing black jacket, blue j...
📍 Location: Golden Gate Park, near the jogging trail, San Francisco
📅 Sighting Date: 2024-08-21 at 08:30:00
📞 Witness: Maria Rodriguez
⭐ Confidence Level: High
🚨 Source Type: Witness
🔴 Priority: High
✅ Verified: False


In [24]:
# Add a sighting that can be linked to the missing person
SIGHTING_INSERT_QUERY = f"""
MERGE `{DATASET_ID}.sightings` AS target
USING (
  SELECT
    @id AS id,
    @sighting_number AS sighting_number,
    @description AS description,
    @sighted_date AS sighted_date,
    @sighted_time AS sighted_time,
    @sighted_address AS sighted_address,
    @sighted_city AS sighted_city,
    @sighted_country AS sighted_country,
    @sighted_postal_code AS sighted_postal_code,
    @sighted_latitude AS sighted_latitude,
    @sighted_longitude AS sighted_longitude,
    CASE
      WHEN @sighted_latitude IS NOT NULL AND @sighted_longitude IS NOT NULL
      THEN ST_GEOGPOINT(@sighted_longitude, @sighted_latitude)
      ELSE NULL
    END AS sighted_geo,
    @apparent_gender AS apparent_gender,
    @apparent_age_range AS apparent_age_range,
    @height_estimate AS height_estimate,
    @hair_color AS hair_color,
    @clothing_description AS clothing_description,
    @distinguishing_features AS distinguishing_features,
    @source_type AS source_type,
    @witness_name AS witness_name,
    @witness_phone AS witness_phone,
    @witness_email AS witness_email,
    @circumstances AS circumstances,
    @confidence_level AS confidence_level,
    @photo_url AS photo_url,
    @status AS status,
    @priority AS priority,
    CAST(@verified AS BOOL) AS verified,
    @notes AS notes,
    CURRENT_TIMESTAMP() AS created_date,
    CURRENT_TIMESTAMP() AS updated_date,
    AI.GENERATE(
      CONCAT(
        'Generate a comprehensive summary paragraph for this sighting report for law enforcement analysis and matching purposes. ',
        'Write it as a single, flowing, discursive paragraph without bullet points, lists, or structured formatting. ',
        'Include key identifying features, location details in narrative form. ',
        'Return only the summary paragraph without any introduction, conclusion, or additional commentary from the model. ',
        'Sighting: ', @description, '. ',
        'Observed on ', CAST(@sighted_date AS STRING),
        CASE
          WHEN @sighted_time IS NOT NULL THEN CONCAT(' at ', CAST(@sighted_time AS STRING))
          ELSE ''
        END,
        ' in ', @sighted_city, ', ', @sighted_country, '. ',
        'Location: ', @sighted_address,
        CASE
          WHEN @sighted_postal_code IS NOT NULL THEN CONCAT(', ', @sighted_postal_code)
          ELSE ''
        END,
        '. Circumstances: ', @circumstances, '. ',
        CASE
          WHEN @notes IS NOT NULL THEN CONCAT('Additional notes: ', @notes, '. ')
          ELSE ''
        END,
        'Status: ', @status, ', Priority: ', @priority, '.'
      ),
      connection_id => '{PROJECT_ID}.{LOCATION}.{CONNECTION_ID}',
      endpoint => 'gemini-2.5-flash',
      model_params => JSON '{{"generation_config": {{"temperature": 0}}}}'
    ).result AS ml_summary
) AS source
ON target.id = source.id
WHEN NOT MATCHED THEN
  INSERT (
    id, sighting_number, description, sighted_date, sighted_time,
    sighted_address, sighted_city, sighted_country, sighted_postal_code,
    sighted_latitude, sighted_longitude, sighted_geo,
    apparent_gender, apparent_age_range, height_estimate, hair_color,
    clothing_description, distinguishing_features,
    source_type, witness_name, witness_phone, witness_email,
    circumstances, confidence_level, photo_url,
    status, priority, verified, notes,
    created_date, updated_date, ml_summary
  )
  VALUES (
    source.id, source.sighting_number, source.description, source.sighted_date, source.sighted_time,
    source.sighted_address, source.sighted_city, source.sighted_country, source.sighted_postal_code,
    source.sighted_latitude, source.sighted_longitude, source.sighted_geo,
    source.apparent_gender, source.apparent_age_range, source.height_estimate, source.hair_color,
    source.clothing_description, source.distinguishing_features,
    source.source_type, source.witness_name, source.witness_phone, source.witness_email,
    source.circumstances, source.confidence_level, source.photo_url,
    source.status, source.priority, source.verified, source.notes,
    source.created_date, source.updated_date, source.ml_summary
  );
"""

# Query to verify the record
VERIFY_SIGHTING_RECORD_QUERY = f"""
SELECT
  id,
  sighting_number,
  description,
  sighted_date,
  sighted_time,
  sighted_city,
  apparent_gender,
  apparent_age_range,
  height_estimate,
  clothing_description,
  distinguishing_features,
  source_type,
  witness_name,
  confidence_level,
  status,
  priority,
  verified,
  ml_summary
FROM
  `{DATASET_ID}.sightings`
WHERE
  id = @sighting_id
LIMIT 1
"""

In [25]:
# Execute the sighting insert query
try:
    print("Inserting sighting record with AI-generated summary...")

    # Configure the query job with parameters for sighting
    sighting_job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("id", "STRING", sample_sighting["id"]),
            bigquery.ScalarQueryParameter(
                "sighting_number", "STRING", sample_sighting["sighting_number"]
            ),
            bigquery.ScalarQueryParameter(
                "description", "STRING", sample_sighting["description"]
            ),
            bigquery.ScalarQueryParameter(
                "sighted_date", "DATE", sample_sighting["sighted_date"]
            ),
            bigquery.ScalarQueryParameter(
                "sighted_time", "TIME", sample_sighting["sighted_time"]
            ),
            bigquery.ScalarQueryParameter(
                "sighted_address", "STRING", sample_sighting["sighted_address"]
            ),
            bigquery.ScalarQueryParameter(
                "sighted_city", "STRING", sample_sighting["sighted_city"]
            ),
            bigquery.ScalarQueryParameter(
                "sighted_country", "STRING", sample_sighting["sighted_country"]
            ),
            bigquery.ScalarQueryParameter(
                "sighted_postal_code",
                "STRING",
                sample_sighting["sighted_postal_code"],
            ),
            bigquery.ScalarQueryParameter(
                "sighted_latitude", "FLOAT64", sample_sighting["sighted_latitude"]
            ),
            bigquery.ScalarQueryParameter(
                "sighted_longitude", "FLOAT64", sample_sighting["sighted_longitude"]
            ),
            # Person Description fields
            bigquery.ScalarQueryParameter(
                "apparent_gender", "STRING", sample_sighting["apparent_gender"]
            ),
            bigquery.ScalarQueryParameter(
                "apparent_age_range", "STRING", sample_sighting["apparent_age_range"]
            ),
            bigquery.ScalarQueryParameter(
                "height_estimate", "FLOAT64", sample_sighting["height_estimate"]
            ),
            bigquery.ScalarQueryParameter(
                "hair_color", "STRING", sample_sighting["hair_color"]
            ),
            bigquery.ScalarQueryParameter(
                "clothing_description", "STRING", sample_sighting["clothing_description"]
            ),
            bigquery.ScalarQueryParameter(
                "distinguishing_features", "STRING", sample_sighting["distinguishing_features"]
            ),
            # Source Information fields
            bigquery.ScalarQueryParameter(
                "source_type", "STRING", sample_sighting["source_type"]
            ),
            bigquery.ScalarQueryParameter(
                "witness_name", "STRING", sample_sighting["witness_name"]
            ),
            bigquery.ScalarQueryParameter(
                "witness_phone", "STRING", sample_sighting["witness_phone"]
            ),
            bigquery.ScalarQueryParameter(
                "witness_email", "STRING", sample_sighting["witness_email"]
            ),
            # Sighting Details
            bigquery.ScalarQueryParameter(
                "circumstances", "STRING", sample_sighting["circumstances"]
            ),
            bigquery.ScalarQueryParameter(
                "confidence_level", "STRING", sample_sighting["confidence_level"]
            ),
            bigquery.ScalarQueryParameter(
                "photo_url", "STRING", sample_sighting["photo_url"]
            ),
            # Status and Processing fields
            bigquery.ScalarQueryParameter(
                "status", "STRING", sample_sighting["status"]
            ),
            bigquery.ScalarQueryParameter(
                "priority", "STRING", sample_sighting["priority"]
            ),
            # Convert Python boolean to BigQuery boolean string
            bigquery.ScalarQueryParameter(
                "verified", "STRING", "TRUE" if sample_sighting["verified"] else "FALSE"
            ),
            bigquery.ScalarQueryParameter(
                "notes", "STRING", sample_sighting["notes"]
            ),
        ]
    )

    # Execute the sighting insert query
    query_job = client.query(SIGHTING_INSERT_QUERY, job_config=sighting_job_config)
    results = query_job.result()  # Wait for the query to complete

    print("✅ Sighting record inserted successfully with AI-generated summary!")
    print(f"🔑 Sighting ID: {sample_sighting['id']}")
    print(f"📋 Sighting Number: {sample_sighting['sighting_number']}")
    print(f"👁️ Description: {sample_sighting['description'][:100]}...")
    print(f"📍 Location: {sample_sighting['sighted_city']}")
    print(f"🚨 Source Type: {sample_sighting['source_type']}")
    print(f"🔴 Priority: {sample_sighting['priority']}")
    print(f"✅ Verified: {sample_sighting['verified']}")
    print(f"🔧 Query job completed: {query_job.job_id}")

    # Verify the inserted record
    verify_job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter(
                "sighting_id", "STRING", sample_sighting["id"]
            )
        ]
    )

    verify_job = client.query(VERIFY_SIGHTING_RECORD_QUERY, job_config=verify_job_config)
    verify_results = verify_job.result()

    if verify_results.total_rows > 0:
        print("\n" + "=" * 80)
        print("INSERTED SIGHTING RECORD WITH AI-GENERATED SUMMARY")
        print("=" * 80)

        for row in verify_results:
            print(f"📋 Sighting ID: {row.id}")
            print(f"📋 Sighting Number: {row.sighting_number}")
            print(f"👁️ Description: {row.description}")
            print(f"📅 Date/Time: {row.sighted_date} at {row.sighted_time}")
            print(f"📍 Location: {row.sighted_city}")
            print(f"👤 Person Details: {row.apparent_gender}, {row.apparent_age_range}")
            print(f"📏 Height: {row.height_estimate}cm")
            print(f"👕 Clothing: {row.clothing_description}")
            print(f"🔍 Features: {row.distinguishing_features}")
            print(f"🚨 Source: {row.source_type}")
            print(f"📞 Witness: {row.witness_name}")
            print(f"⭐ Confidence: {row.confidence_level}")
            print(f"🔴 Priority: {row.priority}")
            print(f"📊 Status: {row.status}")
            print(f"✅ Verified: {row.verified}")

            print("\n" + "-" * 80)
            print("🤖 AI-GENERATED SIGHTING SUMMARY:")
            print("-" * 80)
            print(f"{row.ml_summary}")
            print("-" * 80)

            # Calculate summary statistics
            summary_length = len(row.ml_summary) if row.ml_summary else 0
            word_count = len(row.ml_summary.split()) if row.ml_summary else 0
            print("\n📊 Summary Statistics:")
            print(f"   • Length: {summary_length} characters")
            print(f"   • Word count: {word_count} words")
            print(f"   • Format: {'Discursive paragraph' if summary_length > 100 else 'Short summary'}")

except Exception as e:
    print(f"❌ Error executing sighting insert query: {str(e)}")
    print(f"Error type: {type(e).__name__}")
    if hasattr(e, "errors") and e.errors:
        for error in e.errors:
            print(f"Error details: {error}")

Inserting sighting record with AI-generated summary...
✅ Sighting record inserted successfully with AI-generated summary!
🔑 Sighting ID: 5945a5b3-9063-46b8-b5e8-63213571a028
📋 Sighting Number: SIGHT-2024-070
👁️ Description: 30 to 35-year-old male jogger, tall, near 70kg, brown hair, green eyes, wearing black jacket, blue j...
📍 Location: San Francisco
🚨 Source Type: Witness
🔴 Priority: High
✅ Verified: False
🔧 Query job completed: d87f0dec-4590-41c9-a4ed-48fc523935d5

INSERTED SIGHTING RECORD WITH AI-GENERATED SUMMARY
📋 Sighting ID: 5945a5b3-9063-46b8-b5e8-63213571a028
📋 Sighting Number: SIGHT-2024-070
👁️ Description: 30 to 35-year-old male jogger, tall, near 70kg, brown hair, green eyes, wearing black jacket, blue jeans, white sneakers, has distinctive tattoo on right arm, appeared disoriented, matches missing diabetic runner from San Francisco
📅 Date/Time: 2024-08-21 at 08:30:00
📍 Location: San Francisco
👤 Person Details: Male, 30-40
📏 Height: 175.0cm
👕 Clothing: Black jacket, blue j

# 🧠 AI embedding generation

This section demonstrates the creation of embeddings from missing and sighings summaries through BigQuery. Embeddings are numerical representations that capture the characteristics of provided data, enabling similarity matching and semantic search capabilities.

🤖 **Technical process:**
- Stores embeddings in BigQuery for scalable similarity search
- Enables semantic matching beyond simple textual comparison


In [26]:
# Calculate embeddings for missing persons table
UPDATE_MISSING_PERSONS_EMBEDDINGS_QUERY = f"""
UPDATE `{DATASET_ID}.missing_persons` AS mp
SET mp.ml_summary_embedding = e.ml_generate_embedding_result
FROM ML.GENERATE_EMBEDDING(
    MODEL `{DATASET_ID}.text_embedding_model`,
    (SELECT id, ml_summary as content FROM `{DATASET_ID}.missing_persons` WHERE ml_summary IS NOT NULL AND (ml_summary_embedding IS NULL OR ARRAY_LENGTH(ml_summary_embedding) = 0)),
    STRUCT('SEMANTIC_SIMILARITY' as task_type)
) as e
WHERE mp.id = e.id;
"""

# Calculate embeddings for sightings table
UPDATE_SIGHTINGS_EMBEDDINGS_QUERY = f"""
UPDATE `{DATASET_ID}.sightings` as s
SET s.ml_summary_embedding = e.ml_generate_embedding_result
FROM ML.GENERATE_EMBEDDING(
    MODEL `{DATASET_ID}.text_embedding_model`,
    (SELECT id, ml_summary as content FROM `{DATASET_ID}.sightings` WHERE ml_summary IS NOT NULL AND (ml_summary_embedding IS NULL OR ARRAY_LENGTH(ml_summary_embedding) = 0)),
    STRUCT('SEMANTIC_SIMILARITY' as task_type)
) as e
WHERE e.id = s.id;
"""

print("✅ Embedding update queries prepared!")
print("🧠 These queries will calculate embeddings for:")
print("   • Missing persons table (id, ml_summary field)")
print("   • Sightings table (id, ml_summary field)")
print(f"📊 Using the BigQuery ML text embedding model: {DATASET_ID}.text_embedding_model")

✅ Embedding update queries prepared!
🧠 These queries will calculate embeddings for:
   • Missing persons table (id, ml_summary field)
   • Sightings table (id, ml_summary field)
📊 Using the BigQuery ML text embedding model: homeward.text_embedding_model


In [27]:
VERIFY_EMBEDDING_QUERY = f"""
SELECT
    'missing_persons' as table_name,
    COUNT(*) as total_records,
    COUNT(ml_summary_embedding) as records_with_embeddings,
    ROUND(COUNT(ml_summary_embedding) * 100.0 / COUNT(*), 2) as embedding_coverage_pct
FROM `{DATASET_ID}.missing_persons`
UNION ALL
SELECT
    'sightings' as table_name,
    COUNT(*) as total_records,
    COUNT(ml_summary_embedding) as records_with_embeddings,
    ROUND(COUNT(ml_summary_embedding) * 100.0 / COUNT(*), 2) as embedding_coverage_pct
FROM `{DATASET_ID}.sightings`
"""

In [28]:
# Execute embedding calculations for both tables
try:
    print("Calculating embeddings for missing persons table...")

    # Update missing persons embeddings
    mp_job = client.query(UPDATE_MISSING_PERSONS_EMBEDDINGS_QUERY)
    mp_results = mp_job.result()
    print(
        f"✅ Missing persons embeddings updated! Rows modified: {mp_job.num_dml_affected_rows}"
    )

    print("\nCalculating embeddings for sightings table...")

    # Update sightings embeddings
    sightings_job = client.query(UPDATE_SIGHTINGS_EMBEDDINGS_QUERY)
    sightings_results = sightings_job.result()
    print(
        f"✅ Sightings embeddings updated! Rows modified: {sightings_job.num_dml_affected_rows}"
    )

except Exception as e:
    print(f"❌ Error calculating embeddings: {str(e)}")
    print(f"Error type: {type(e).__name__}")
    if hasattr(e, "errors") and e.errors:
        for error in e.errors:
            print(f"Error details: {error}")

Calculating embeddings for missing persons table...
✅ Missing persons embeddings updated! Rows modified: 1

Calculating embeddings for sightings table...
✅ Sightings embeddings updated! Rows modified: 1


In [29]:
try:

    # Verify embeddings were created
    verify_job = client.query(VERIFY_EMBEDDING_QUERY)
    verify_results = verify_job.result()

    print("\n" + "=" * 80)
    print("EMBEDDING CALCULATION SUMMARY")
    print("=" * 80)

    for row in verify_results:
        print(f"📊 Table: {row.table_name}")
        print(f"   Total Records: {row.total_records}")
        print(f"   Records with Embeddings: {row.records_with_embeddings}")
        print(f"   Coverage: {row.embedding_coverage_pct}%")
        print()

except Exception as e:
    print(f"❌ Error checking embeddings: {str(e)}")
    print(f"Error type: {type(e).__name__}")
    if hasattr(e, "errors") and e.errors:
        for error in e.errors:
            print(f"Error details: {error}")


EMBEDDING CALCULATION SUMMARY
📊 Table: missing_persons
   Total Records: 1
   Records with Embeddings: 1
   Coverage: 100.0%

📊 Table: sightings
   Total Records: 1
   Records with Embeddings: 1
   Coverage: 100.0%



---
# 📊 Vector Search Index Optimization

**⚠️ Performance optimization - Run only if you have imported more than 5,000 examples (importing your custom data) ⚠️**

This section creates vector indexes to accelerate similarity search operations when working with large datasets. Vector indexes dramatically improve query performance.

In [ ]:
MISSING_PERSON_VECTOR_INDEX = f"""
CREATE OR REPLACE VECTOR INDEX missing_person_v_index
ON `{DATASET_ID}.missing_persons`(ml_summary_embedding)
OPTIONS(
  index_type = 'IVF',
  distance_type = 'COSINE',
  ivf_options = '{{"num_lists":500}}'
)
"""

SIGHTING_VECTOR_INDEX = f"""
CREATE OR REPLACE VECTOR INDEX sighting_v_index
ON `{DATASET_ID}.missing_persons`(ml_summary_embedding)
OPTIONS(
  index_type = 'IVF',
  distance_type = 'COSINE',
  ivf_options = '{{"num_lists":500}}'
)
"""

# Execute vector index creation queries
try:
    print("Creating vector index for missing persons table...")
    
    # Create missing persons vector index
    mp_index_job = client.query(MISSING_PERSON_VECTOR_INDEX)
    mp_index_results = mp_index_job.result()
    print("✅ Missing persons vector index created successfully!")
    
    print("\nCreating vector index for sightings table...")
    
    # Create sightings vector index
    sightings_index_job = client.query(SIGHTING_VECTOR_INDEX)
    sightings_index_results = sightings_index_job.result()
    print("✅ Sightings vector index created successfully!")
    
    print("\n🎯 Tables are now optimized for fast similarity searches")
    
except Exception as e:
    print(f"❌ Error creating vector indexes: {str(e)}")
    print(f"Error type: {type(e).__name__}")
    if hasattr(e, 'errors') and e.errors:
        for error in e.errors:
            print(f"Error details: {error}")

---
# 🔍 Find Sightings by Missing Person (AI Similarity Search)

This section demonstrates how to search for potential sightings of a specific missing person using similarity matching. The query compares the missing person's AI-generated summary against all recorded sightings in the same area to identify possible matches.

🎯 **Search methodology:**
- Compares missing person description embeddings with sighting embeddings, filtering by geographic proximity and time constraints
- Calculates similarity scores using vector similarity
- Ranks results by confidence level and temporal relevance

In [30]:
# Similarity Search 1: Find sightings that match a missing person
SIMILARITY_SEARCH_MP_TO_SIGHTINGS_QUERY = f"""
-- Search sightings that are similar to a specific missing person with geo and time filtering
SELECT
query.id,
query.case_number,
distance,
base.id,
base.sighting_number,
base.sighted_date,
base.sighted_time,
base.sighted_city,
base.sighted_geo,
base.witness_name,
base.confidence_level,
base.ml_summary
FROM
  VECTOR_SEARCH(
    (
      SELECT *
      FROM 
        `{DATASET_ID}.sightings`
      WHERE
        DATE(created_date) >= DATE_SUB(@last_seen_date, INTERVAL @delta_days DAY)
    ),
    'ml_summary_embedding',
    (SELECT id, case_number, ml_summary_embedding FROM `{DATASET_ID}.missing_persons` WHERE id = @missing_person_id), 
    top_k => 5,
    distance_type => 'COSINE',
    options => '{{"fraction_lists_to_search": 0.005}}')
WHERE ST_DWITHIN(
  base.sighted_geo, 
  ST_GEOGPOINT(@last_seen_longitude, @last_seen_latitude), 
  @search_radius_meters
);

"""

print("🔍 This query will find sightings that match our missing person John Doe")
print("📊 Results include:")
print("   • Cosine distance (0-2, lower = better match)")
print("   • Top 5 most similar sightings")
print("   • Geo filtering based on configurable search radius from last seen location")
print("   • Time filtering to search from delta_days before last_seen_date to 30 days after based on created_date")

🔍 This query will find sightings that match our missing person John Doe
📊 Results include:
   • Cosine distance (0-2, lower = better match)
   • Top 5 most similar sightings
   • Geo filtering based on configurable search radius from last seen location
   • Time filtering to search from delta_days before last_seen_date to 30 days after based on created_date


In [31]:
# Execute similarity search: Missing Person → Sightings
try:
    print("🔍 Searching for sightings that match our missing person John Doe...")

    # Configure query with missing person ID parameter and geo filtering
    search_job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter(
                "missing_person_id", "STRING", sample_missing_person["id"]
            ),
            bigquery.ScalarQueryParameter(
                "last_seen_latitude", "FLOAT64", sample_missing_person["last_seen_latitude"]
            ),
            bigquery.ScalarQueryParameter(
                "last_seen_longitude", "FLOAT64", sample_missing_person["last_seen_longitude"]
            ),
            bigquery.ScalarQueryParameter(
                "search_radius_meters", "FLOAT64", 10000.0  # 10km search radius
            ),
            bigquery.ScalarQueryParameter(
                "last_seen_date", "DATE", sample_missing_person["last_seen_date"]
            ),
            bigquery.ScalarQueryParameter(
                "delta_days", "INT64", 7  # Search 7 days before last seen date
            ),
        ]
    )

    # Execute the similarity search query
    search_job = client.query(
        SIMILARITY_SEARCH_MP_TO_SIGHTINGS_QUERY, job_config=search_job_config
    )
    search_results = search_job.result()

    print(
        f"✅ Similarity search completed! Found {search_results.total_rows} potential matches"
    )

    if search_results.total_rows > 0:
        print("\n" + "=" * 100)
        print("SIMILARITY SEARCH RESULTS: MISSING PERSON → SIGHTINGS")
        print("=" * 100)
        print(
            f"🔍 Searching for: {sample_missing_person['name']} {sample_missing_person['surname']} (Case: {sample_missing_person['case_number']})"
        )
        print("=" * 100)

        for i, row in enumerate(search_results, 1):
            print(f"\n🎯 MATCH #{i}")
            print("-" * 50)
            print(
                f"📋 Sighting: {row.sighting_number} | 📏 Distance: {row.distance}"
            )
            print(f"📅 Date/Time: {row.sighted_date} at {row.sighted_time}")
            print(f"📍 Location: {row.sighted_city}")
            print(
                f"📞 Witness: {row.witness_name} | ⭐ Reliability: {row.confidence_level}"
            )

            print("\n🤖 Sighting Summary:")
            print(f"   {row.ml_summary[:150]}...")

            if row.distance <= 0.1:
                print("🚨 HIGH PRIORITY MATCH - Recommend immediate investigation!")
            elif row.distance <= 0.3:
                print("⚠️ GOOD MATCH - Worth investigating")

            print("-" * 50)
    else:
        print("ℹ️ No sighting matches found for this missing person")

except Exception as e:
    print(f"❌ Error executing similarity search: {str(e)}")
    print(f"Error type: {type(e).__name__}")
    if hasattr(e, "errors") and e.errors:
        for error in e.errors:
            print(f"Error details: {error}")

🔍 Searching for sightings that match our missing person John Doe...
✅ Similarity search completed! Found 1 potential matches

SIMILARITY SEARCH RESULTS: MISSING PERSON → SIGHTINGS
🔍 Searching for: John Doe (Case: CASE-2025-0901-0001)

🎯 MATCH #1
--------------------------------------------------
📋 Sighting: SIGHT-2024-070 | 📏 Distance: 0.10458744903004302
📅 Date/Time: 2024-08-21 at 08:30:00
📍 Location: San Francisco
📞 Witness: Maria Rodriguez | ⭐ Reliability: High

🤖 Sighting Summary:
   On August 21, 2024, at approximately 08:30:00, a tall male jogger, estimated to be 30 to 35 years old and weighing around 70kg, with brown hair and gr...
⚠️ GOOD MATCH - Worth investigating
--------------------------------------------------


# 🔄 Identify missing persons from sightings (reverse similarity search)

This section demonstrates the reverse search capability - identifying which missing persons might match a newly reported sighting. 

🔄 **Reverse matching process:**
- Takes the embedding of a sighting description and compares against all missing person cases that have been reported in compatible area and time

📊 **Output provides:** Ranked list of possible missing person matches with confidence scores, case details, and investigative contact information.

In [32]:
# Similarity Search: Find missing persons that match a sighting
SIMILARITY_SEARCH_SIGHTINGS_TO_MP_QUERY = f"""
-- Search missing persons that are similar to a specific sighting with geo and time filtering
SELECT
query.id,
query.sighting_number,
distance,
base.id,
base.case_number,
base.name,
base.surname,
base.ml_summary
FROM
  VECTOR_SEARCH(
    (
      SELECT *
      FROM 
        `{DATASET_ID}.missing_persons`
      WHERE
        DATE(created_date) >= DATE_SUB(@sighted_date, INTERVAL @delta_days DAY)
    ),
    'ml_summary_embedding',
    (SELECT id, sighting_number, ml_summary_embedding FROM `{DATASET_ID}.sightings` WHERE id = @sighting_id), 
    top_k => 10,
    distance_type => 'COSINE',
    options => '{{"fraction_lists_to_search": 0.005}}')
WHERE ST_DWITHIN(
  base.last_seen_geo, 
  ST_GEOGPOINT(@sighted_longitude, @sighted_latitude), 
  @search_radius_meters
);
"""

print("🔍 This query will find missing persons that match our sighting")
print("📊 Results include:")
print("   • Cosine distance (0-2, lower = better match)")
print("   • Top 10 most similar missing persons")
print("   • Geo filtering based on configurable search radius from sighting location")
print("   • Time filtering to search from delta_days before sighted_date to 30 days after based on created_date")

🔍 This query will find missing persons that match our sighting
📊 Results include:
   • Cosine distance (0-2, lower = better match)
   • Top 10 most similar missing persons
   • Geo filtering based on configurable search radius from sighting location
   • Time filtering to search from delta_days before sighted_date to 30 days after based on created_date


In [33]:
# Execute similarity search: Sighting → Missing Persons
try:
    print("🔍 Searching for missing persons that match our sighting...")

    # Configure query with sighting ID parameter and geo filtering
    reverse_search_job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter(
                "sighting_id", "STRING", sample_sighting["id"]
            ),
            bigquery.ScalarQueryParameter(
                "sighted_latitude", "FLOAT64", sample_sighting["sighted_latitude"]
            ),
            bigquery.ScalarQueryParameter(
                "sighted_longitude", "FLOAT64", sample_sighting["sighted_longitude"]
            ),
            bigquery.ScalarQueryParameter(
                "search_radius_meters", "FLOAT64", 10000.0  # 10km search radius
            ),
            bigquery.ScalarQueryParameter(
                "sighted_date", "DATE", sample_sighting["sighted_date"]
            ),
            bigquery.ScalarQueryParameter(
                "delta_days", "INT64", 7  # Search 7 days before sighted date
            ),
        ]
    )

    # Execute the reverse similarity search query
    reverse_search_job = client.query(
        SIMILARITY_SEARCH_SIGHTINGS_TO_MP_QUERY, job_config=reverse_search_job_config
    )
    reverse_search_results = reverse_search_job.result()

    print(
        f"✅  Similarity search completed! Found {reverse_search_results.total_rows} potential matches"
    )

    if reverse_search_results.total_rows > 0:
        print("\n" + "=" * 100)
        print("SIMILARITY SEARCH RESULTS: SIGHTING → MISSING PERSONS")
        print("=" * 100)
        print(
            f"🔍 Analyzing sighting: {sample_sighting['sighting_number']}"
        )
        print("=" * 100)

        for i, row in enumerate(reverse_search_results, 1):
            print(f"\n🎯 MATCH #{i}")
            print("-" * 50)
            print(
                f"📋 Case: {row.case_number} | 📏 Distance: {row.distance}"
            )
            print(f"👤 Missing Person: {row.name} {row.surname}")

            print("\n🤖 Missing Person Summary:")
            print(f"   {row.ml_summary[:150]}...")

            if row.distance <= 0.1:
                print("🚨 HIGH PRIORITY MATCH - Strong potential identification!")
            elif row.distance <= 0.3:
                print("⚠️ GOOD MATCH - Worth investigating further")

            print("-" * 50)
    else:
        print("ℹ️ No missing person matches found for this sighting")

except Exception as e:
    print(f"❌ Error executing reverse similarity search: {str(e)}")
    print(f"Error type: {type(e).__name__}")
    if hasattr(e, "errors") and e.errors:
        for error in e.errors:
            print(f"Error details: {error}")

🔍 Searching for missing persons that match our sighting...
✅  Similarity search completed! Found 1 potential matches

SIMILARITY SEARCH RESULTS: SIGHTING → MISSING PERSONS
🔍 Analyzing sighting: SIGHT-2024-070

🎯 MATCH #1
--------------------------------------------------
📋 Case: CASE-2025-0901-0001 | 📏 Distance: 0.10458744903004302
👤 Missing Person: John Doe

🤖 Missing Person Summary:
   John Doe, a 35-year-old male, was last seen on September 1, 2025, at approximately 18:45:00 in San Francisco, USA, specifically at 456 Oak Avenue, 941...
⚠️ GOOD MATCH - Worth investigating further
--------------------------------------------------


# 📹 AI-Powered Surveillance Video Analysis
This section showcases the demo most advanced capability - automatically analyzing surveillance camera footage to detect and match missing persons. Using Google's Gemini multimodal AI, the system can process video content and identify potential matches.

🎥 **Video analysis capabilities:**
- Processes surveillance footage from multiple camera sources
- Extracts individual frames and identifies human figures
- Compares detected persons against missing person databases
- Generates confidence scores and location metadata

🏙️ **Smart city integration:**
This technology enables law enforcement to leverage existing surveillance infrastructure across entire metropolitan areas, automatically scanning thousands of hours of footage for missing persons.

⚡ **Real-world impact:** Transforms passive surveillance systems into active search tools, dramatically expanding the search radius and reducing time to locate missing individuals.

⏱️ **Processing speed:** Analyzes video content at 2-5x real-time speed, depending on resolution and system load.

# ⬇️ Recordings download

This section demonstrates the video ingestion pipeline of the missing persons detection system by using a Free dataset that contains recordings. For this specific scenario, to save time and costs i used the lowest resolution found in the dataset, as a consequence Gemini is not always able to correctly detect the person as described in the above example. For this reason I added also a video i realized myself, but you can use whichever video you prefer.

**What you will find in this section**

1. **Download VIRAT dataset samples**: Retrieve sample surveillance videos from the VIRAT Video and Image Dataset Release 2.0, which contains realistic scenarios of people in various outdoor environments. These videos are temporarily stored locally for processing.

2. **Upload to Google Cloud Storage with metadata enrichment**: Each video is uploaded to the GCS bucket together with metadata headers including camera information, geolocation data, timestamps, and technical specifications.

3. **Create BigQuery External Table**: a BigQuery external table that directly references the video files in GCS, enabling SQL-based querying and Gemini multimodal processing.

*Note: During development, I encountered metadata cache invalidation issues when creating the external table before uploading files. For this reason i decided to upload files first, then create the table, as the external table will already retrieve all the metadata needed by GCS. All encountered limitations and workarounds are documented in our technical limitations document.

In [34]:
VIRAT_VIDEO_SAMPLES = {
  "metadata": {
    "source": "https://data.kitware.com/#collection/56f56db28d777f753209ba9f/folder/56f581ce8d777f753209ca43",
    "dataset": "VIRAT Video and Image Dataset Release 2.0",
    "created_at": "2025-08-18T00:00:00Z",
    "version": "1.0"
  },
  "videos": [
    {
      "id": "56f587eb8d777f753209cc12",
      "download_url": "https://data.kitware.com/api/v1/item/56f587eb8d777f753209cc12/download",
      "camera_id": "CAM001",
      "timestamp": "20240815123000",
      "latitude": 37.7849,
      "longitude": -122.4094,
      "camera_type": "VIRAT_OPENDATA",
      "resolution": "1920x1080",
      "location": "San_Francisco",
      "duration_seconds": 420,
      "mime_type": "video/mp4"
    },
    {
      "id": "56f587ca8d777f753209cbb2",
      "download_url": "https://data.kitware.com/api/v1/item/56f587ca8d777f753209cbb2/download",
      "camera_id": "CAM002",
      "timestamp": "20240815140000",
      "latitude": 37.7849,
      "longitude": -122.4094,
      "camera_type": "VIRAT_OPENDATA",
      "resolution": "1920x1080",
      "location": "San_Francisco",
      "duration_seconds": 360,
      "mime_type": "video/mp4"
    },
    {
      "id": "56f587a98d777f753209cb6d",
      "download_url": "https://data.kitware.com/api/v1/item/56f587a98d777f753209cb6d/download",
      "camera_id": "CAM003",
      "timestamp": "20240815153000",
      "latitude": 37.7849,
      "longitude": -122.4094,
      "camera_type": "VIRAT_OPENDATA",
      "resolution": "1920x1080",
      "location": "San_Francisco",
      "duration_seconds": 480,
      "mime_type": "video/mp4"
    },
    {
      "id": "56f5878c8d777f753209caf2",
      "download_url": "https://data.kitware.com/api/v1/item/56f5878c8d777f753209caf2/download",
      "camera_id": "CAM004",
      "timestamp": "20240815170000",
      "latitude": 37.7849,
      "longitude": -122.4094,
      "camera_type": "VIRAT_OPENDATA",
      "resolution": "1920x1080",
      "location": "San_Francisco",
      "duration_seconds": 390,
      "mime_type": "video/mp4"
    }
  ]
}

In [35]:
print("\n=== Download video samples ===")

# Create temporary directory for downloads
temp_dir = tempfile.mkdtemp()
print(f"Using temporary directory: {temp_dir}")

success_count = 0
failure_count = 0

# Process each video in the VIRAT_VIDEO_SAMPLES
for i, video in enumerate(VIRAT_VIDEO_SAMPLES["videos"], 1):
    video_id = video["id"]
    download_url = video["download_url"]
    camera_id = video["camera_id"]
    timestamp = video["timestamp"]
    latitude = video["latitude"]
    longitude = video["longitude"]
    camera_type = video["camera_type"]
    resolution = video["resolution"]
    mime_type = video["mime_type"]

    # Generate filename according to Homeward naming convention
    filename = f"{camera_id}_{timestamp}_{latitude}_{longitude}_{camera_type}_{resolution}.mp4"
    temp_file = os.path.join(temp_dir, filename)

    print(f"\nProcessing video {i}/{len(VIRAT_VIDEO_SAMPLES['videos'])}: {video_id}")
    print(f"  Filename: {filename}")
    
    # Download video file using curl (similar to setup.sh)
    print(f"  Downloading from: {download_url}")
    download_cmd = f'curl -L -f -o "{temp_file}" "{download_url}" --connect-timeout 30 --max-time 300'

    if run_command(download_cmd):
        # Get file size for reporting
        file_size_cmd = f'du -h "{temp_file}"'
        try:
            result = subprocess.run(file_size_cmd, shell=True, capture_output=True, text=True)
            file_size = result.stdout.split()[0] if result.stdout else "unknown"
            print(f"  Download completed: {file_size}")
        except:
            print("  Download completed")
        
        print("\n=== Upload video samples to GCS ===")
        # Upload video to GCS with proper content type and custom metadata (similar to setup.sh)
        print("  Uploading video to GCS with metadata...") 
        upload_cmd = f"""
gsutil -h "Content-Type:{mime_type}"  \
-h "x-goog-meta-video-id:{video_id}"  \
-h "x-goog-meta-camera-id:{camera_id}"  \
-h "x-goog-meta-timestamp:{timestamp}"  \
-h "x-goog-meta-latitude:{latitude}"  \
-h "x-goog-meta-longitude:{longitude}"  \
-h "x-goog-meta-camera-type:{camera_type}"  \
-h "x-goog-meta-resolution:{resolution}"  \
cp "{temp_file}" "gs://{BUCKET_NAME}/{filename}"
"""

        if run_command(upload_cmd):
            print("  Video uploaded successfully with metadata")
            success_count += 1
        else:
            print(f"  Failed to upload video: {filename}")
            failure_count += 1
            
        # Clean up temporary file
        try:
            os.remove(temp_file)
        except:
            pass
    else:
        print(f"  Failed to download video: {video_id}")
        failure_count += 1

print("\n=== Video Processing Summary ===")
print(f"Successfully processed: {success_count} videos")
if failure_count > 0:
    print(f"Failed to process: {failure_count} videos")

# Clean up temporary directory
try:
    os.rmdir(temp_dir)
except:
    pass

print("\n=== Creating BigQuery External Table ===")
run_command(f"bq mk --table --external_table_definition=\"gs://{BUCKET_NAME}/*.mp4@{LOCATION}.{CONNECTION_ID}\" --object_metadata=SIMPLE \"{PROJECT_ID}:{DATASET_ID}.video_objects\"")



=== Download video samples ===
Using temporary directory: /var/folders/yb/411wr50n6v1cy6bckcv77d7c0000gn/T/tmptob08dta

Processing video 1/4: 56f587eb8d777f753209cc12
  Filename: CAM001_20240815123000_37.7849_-122.4094_VIRAT_OPENDATA_1920x1080.mp4
✅ Success: curl -L -f -o "/var/folders/yb/411wr50n6v1cy6bckcv77d7c0000gn/T/tmptob08dta/CAM001_20240815123000_37.7849_-122.4094_VIRAT_OPENDATA_1920x1080.mp4" "https://data.kitware.com/api/v1/item/56f587eb8d777f753209cc12/download" --connect-timeout 30 --max-time 300
  Download completed: 3.6M

=== Upload video samples to GCS ===
  Uploading video to GCS with metadata...
✅ Success: 
gsutil -h "Content-Type:video/mp4"  -h "x-goog-meta-video-id:56f587eb8d777f753209cc12"  -h "x-goog-meta-camera-id:CAM001"  -h "x-goog-meta-timestamp:20240815123000"  -h "x-goog-meta-latitude:37.7849"  -h "x-goog-meta-longitude:-122.4094"  -h "x-goog-meta-camera-type:VIRAT_OPENDATA"  -h "x-goog-meta-resolution:1920x1080"  cp "/var/folders/yb/411wr50n6v1cy6bckcv77d7c

True

In [36]:
# Advanced AI prompt for video analysis using Gemini's multimodal capabilities
# This prompt template will be filled with actual missing person data for video search

VIDEO_ANALYSIS_PROMPT = """# ROLE AND GOAL
You are a state-of-the-art AI visual analysis system with an expert specialization in human identification within low-quality video footage.
Your primary mission is to analyze the provided video for a critical missing person case with the highest degree of accuracy and diligence.
You must be methodical and detail-oriented in your analysis and reporting.

# TASK CONTEXT
This is a high-priority, time-sensitive analysis.
The provided video is a low-quality security footage from the street, where people are walking around.
The objective is to determine if the missing person is visible in this video, and if so, to extract all relevant information about their presence.

# MISSING PERSON DATA
Carefully analyze the following description of the missing person. Every detail is crucial.

- **Gender:** `{gender}`
- **Approximate Age:** `{age}`
- **Build - Height:** `{build_height}`
- **Hair Color and Style:** `{hair}`
- **Clothing (Top):** `{clothing_top}`
- **Clothing (Bottom):** `{clothing_botton}`
- **Footwear:** `{footwear}`
- **Accessories:** `{accessories}`
- **Distinguishing Features:** `{features}`

# ANALYSIS INSTRUCTIONS
You must perform the following steps in your analysis:

1.  **Full Video Scan:** Meticulously review the entire video from start to finish. Do not stop after a potential first match; the person may appear multiple times.
2.  **Feature Matching:** Compare every individual in the video against the `MISSING PERSON DATA`. Assess matches based on all available criteria: clothing, build, hair, accessories, and any visible distinguishing features or mannerisms.
3.  **Justification:** You MUST provide a step-by-step justification for your match. List the features that matched, the features that did not match, and any features that were ambiguous or obscured (e.g., 'Face was unclear, but clothing is a 90% match').
4.  **Contextual Analysis (If Found):** If you identify the person with confidence:
    -   Note the exact timestamp(s) (in `HH:MM:SS` format) of their appearance.
    -   Describe their actions and behavior (e.g., 'walking quickly', 'talking on the phone', 'sitting on a bench', 'seemed distressed').
    -   Analyze if they are with anyone else. If so, provide a detailed description of each companion (gender, estimated age, clothing, etc.).
    -   Describe their direction of travel.
5. **Confidence Score (If Found):** If you identify a person with confidence:
    -   Return the confidence score of the finding in the range 0.0 to 1.0
"""

In [37]:
# Sample missing person data for video analysis demo
# In the web app, this would be dynamically populated from the missing person record
sample_video_search_person = {
    "gender": "Male",
    "age": "35",
    "build_height": "1.7m, 80kg",
    "hair": "Bald, sporty",
    "clothing_top": "Red shirt",
    "clothing_botton": "Blue pants",
    "footwear": "White sneakers",
    "accessories": "None",
    "features": "None",
}

# Video analysis filtering parameters
VIDEO_ANALYSIS_BQ_START_DATE = "2024-08-01"
VIDEO_ANALYSIS_BQ_END_DATE   = "2025-06-01"
VIDEO_ANALYSIS_BQ_TIME_RANGE = "BETWEEN 11 AND 17" 

# Geographic filtering parameters (Toronto coordinates as example)
VIDEO_ANALYSIS_BQ_LATITUDE = "37.7889"     # Last seen latitude
VIDEO_ANALYSIS_BQ_LONGITUDE = "-122.4084"   # Last seen longitude 

# Test with Toronto
#VIDEO_ANALYSIS_BQ_LATITUDE = "43.6532"     # Last seen latitude
#VIDEO_ANALYSIS_BQ_LONGITUDE = "-79.3832"   # Last seen longitude  
VIDEO_ANALYSIS_BQ_RADIUS_KM = "5.0"        # Search radius in kilometers

# Format the video analysis prompt with the sample person data
VIDEO_ANALYSIS_PROMPT = VIDEO_ANALYSIS_PROMPT.format(
    gender=sample_video_search_person["gender"],
    age=sample_video_search_person["age"],
    build_height=sample_video_search_person["build_height"],
    hair=sample_video_search_person["hair"],
    clothing_top=sample_video_search_person["clothing_top"],
    clothing_botton=sample_video_search_person["clothing_botton"],
    footwear=sample_video_search_person["footwear"],
    accessories=sample_video_search_person["accessories"],
    features=sample_video_search_person["features"],
)

print("🎬 Video analysis prompt configured for missing person search")
print(f"👤 Target: {sample_video_search_person['gender']}, {sample_video_search_person['age']} years old")
print(f"📏 Physical: {sample_video_search_person['build_height']}")
print(f"👕 Clothing: {sample_video_search_person['clothing_top']}, {sample_video_search_person['clothing_botton']}")
print(f"👟 Footwear: {sample_video_search_person['footwear']}")
print(f"🕶️ Accessories: {sample_video_search_person['accessories']}")
print(f"📆 Date Range: From {VIDEO_ANALYSIS_BQ_START_DATE} to {VIDEO_ANALYSIS_BQ_END_DATE}")
print(f"🕐 Time Range: {VIDEO_ANALYSIS_BQ_TIME_RANGE}")
print(f"🌍 Geographic Center: ({VIDEO_ANALYSIS_BQ_LATITUDE}, {VIDEO_ANALYSIS_BQ_LONGITUDE})")
print(f"📍 Search Radius: {VIDEO_ANALYSIS_BQ_RADIUS_KM} km")

🎬 Video analysis prompt configured for missing person search
👤 Target: Male, 35 years old
📏 Physical: 1.7m, 80kg
👕 Clothing: Red shirt, Blue pants
👟 Footwear: White sneakers
🕶️ Accessories: None
📆 Date Range: From 2024-08-01 to 2025-06-01
🕐 Time Range: BETWEEN 11 AND 17
🌍 Geographic Center: (37.7889, -122.4084)
📍 Search Radius: 5.0 km


In [38]:
# Display the complete video analysis prompt for verification
# This shows the exact prompt that will be sent to Gemini for video analysis

print("\n" + "=" * 100)
print("🎬 COMPLETE VIDEO ANALYSIS PROMPT FOR GEMINI")
print("=" * 100)
print(VIDEO_ANALYSIS_PROMPT)
print("=" * 100)


🎬 COMPLETE VIDEO ANALYSIS PROMPT FOR GEMINI
# ROLE AND GOAL
You are a state-of-the-art AI visual analysis system with an expert specialization in human identification within low-quality video footage.
Your primary mission is to analyze the provided video for a critical missing person case with the highest degree of accuracy and diligence.
You must be methodical and detail-oriented in your analysis and reporting.

# TASK CONTEXT
This is a high-priority, time-sensitive analysis.
The provided video is a low-quality security footage from the street, where people are walking around.
The objective is to determine if the missing person is visible in this video, and if so, to extract all relevant information about their presence.

# MISSING PERSON DATA
Carefully analyze the following description of the missing person. Every detail is crucial.

- **Gender:** `Male`
- **Approximate Age:** `35`
- **Build - Height:** `1.7m, 80kg`
- **Hair Color and Style:** `Bald, sporty`
- **Clothing (Top):** `R

In [44]:
# BigQuery query to analyze surveillance videos using Gemini 2.5 Pro multimodal model
# This query processes videos in the {DATASET_ID}.video_objects table with time and geo filtering to
# reduce costs on Gemini API and applies AI analysis to detect the missing person
#     output_schema includes the 'result' field due to a problem with AI.GENERATE func

VIDEO_ANALYSIS_BQ_QUERY = f"""
SELECT
  uri,
  AI.GENERATE(
    (
      "{{VIDEO_ANALYSIS_PROMPT}}",
      "\\n# RECORDING:  ",
      OBJ.GET_ACCESS_URL(ref, 'r')
    ),
    connection_id => '{PROJECT_ID}.{LOCATION}.{CONNECTION_ID}',
    endpoint => 'gemini-2.5-pro',
    output_schema => 'personFound BOOL, confidenceScore FLOAT64,summaryOfFindings STRING',
    model_params => JSON '{{"generation_config": {{"temperature": 0}}}}') as result
FROM `{DATASET_ID}.video_objects`
WHERE 1=1
AND EXISTS (
  SELECT 1 FROM UNNEST(metadata) AS meta
  WHERE meta.name = 'timestamp'
  AND PARSE_DATETIME('%Y%m%d%H%M%S', meta.value) BETWEEN
  DATETIME('{VIDEO_ANALYSIS_BQ_START_DATE}') AND DATETIME('{VIDEO_ANALYSIS_BQ_END_DATE}')
)
AND EXISTS (
  SELECT 1 FROM UNNEST(metadata) AS meta
  WHERE meta.name = 'timestamp'
  AND EXTRACT(HOUR FROM PARSE_DATETIME('%Y%m%d%H%M%S', meta.value)) {VIDEO_ANALYSIS_BQ_TIME_RANGE}
)
AND ST_DWITHIN(
  ST_GEOGPOINT(
    CAST((SELECT value FROM UNNEST(metadata) WHERE name = 'longitude') AS FLOAT64),
    CAST((SELECT value FROM UNNEST(metadata) WHERE name = 'latitude') AS FLOAT64)
  ),
  ST_GEOGPOINT({VIDEO_ANALYSIS_BQ_LONGITUDE}, {VIDEO_ANALYSIS_BQ_LATITUDE}),
  {VIDEO_ANALYSIS_BQ_RADIUS_KM} * 1000  -- Convert km to meters for ST_DWITHIN
)
;
""".replace(
    "{VIDEO_ANALYSIS_PROMPT}",
    VIDEO_ANALYSIS_PROMPT.encode("unicode-escape")
    .replace(b'"', b'\\"')
    .decode("utf-8"),
).replace(
    "{VIDEO_ANALYSIS_BQ_START_DATE}",
    VIDEO_ANALYSIS_BQ_START_DATE
).replace(
    "{VIDEO_ANALYSIS_BQ_END_DATE}",
    VIDEO_ANALYSIS_BQ_END_DATE
).replace(
    "{VIDEO_ANALYSIS_BQ_TIME_RANGE}",
    VIDEO_ANALYSIS_BQ_TIME_RANGE
).replace(
    "{VIDEO_ANALYSIS_BQ_LATITUDE}",
    VIDEO_ANALYSIS_BQ_LATITUDE
).replace(
    "{VIDEO_ANALYSIS_BQ_LONGITUDE}",
    VIDEO_ANALYSIS_BQ_LONGITUDE
).replace(
    "{VIDEO_ANALYSIS_BQ_RADIUS_KM}",
    VIDEO_ANALYSIS_BQ_RADIUS_KM
)

print("🔍 Video analysis BigQuery prepared!")
print("📊 Query features:")
print("   • Processes all videos in {DATASET_ID}.video_objects table")
print("   • Uses Gemini 2.5 Pro multimodal AI model")
print("   • Generates secure access URLs for video analysis")
print("   • Returns structured JSON analysis results")
print("   • Temperature set to 0 for consistent results")
print("   • Date and time filtering using metadata fields")
print("   • Filters by timestamp metadata (format: YYYYMMDDHHMMSS)")
print("   • Geographic filtering using BigQuery ST_GEOGPOINT/ST_DWITHIN functions")
print("   • Efficient geospatial filtering with WGS84 ellipsoid accuracy")
print(f"   • Current filters: {VIDEO_ANALYSIS_BQ_START_DATE} to {VIDEO_ANALYSIS_BQ_END_DATE}")
print(f"   • Time range: Hours {VIDEO_ANALYSIS_BQ_TIME_RANGE}")
print(f"   • Geographic center: ({VIDEO_ANALYSIS_BQ_LATITUDE}, {VIDEO_ANALYSIS_BQ_LONGITUDE})")
print(f"   • Search radius: {VIDEO_ANALYSIS_BQ_RADIUS_KM} km")

🔍 Video analysis BigQuery prepared!
📊 Query features:
   • Processes all videos in {DATASET_ID}.video_objects table
   • Uses Gemini 2.5 Pro multimodal AI model
   • Generates secure access URLs for video analysis
   • Returns structured JSON analysis results
   • Temperature set to 0 for consistent results
   • Date and time filtering using metadata fields
   • Filters by timestamp metadata (format: YYYYMMDDHHMMSS)
   • Geographic filtering using BigQuery ST_GEOGPOINT/ST_DWITHIN functions
   • Efficient geospatial filtering with WGS84 ellipsoid accuracy
   • Current filters: 2024-08-01 to 2025-06-01
   • Time range: Hours BETWEEN 11 AND 17
   • Geographic center: (37.7889, -122.4084)
   • Search radius: 5.0 km


In [45]:
# Execute AI-powered video analysis across all surveillance footage
# This processes each video through Gemini 2.5 Pro for missing person detection

try:
    print("🎬 Executing AI video analysis across all surveillance footage...")
    print("🤖 Using Gemini 2.5 Pro multimodal model for person detection")
    
    # Execute the video analysis query
    query_job = client.query(VIDEO_ANALYSIS_BQ_QUERY)
    results = query_job.result()
    # Collect video URIs where personFound is True
    videos_with_persons = []

    print("✅ Video analysis completed successfully!")
    print(f"📊 Total videos processed: {results.total_rows}")
    print(f"🔧 Query job ID: {query_job.job_id}")

    if results.total_rows > 0:
        print("\n" + "=" * 100)
        print("🎬 SURVEILLANCE VIDEO ANALYSIS RESULTS")
        print("=" * 100)
        
        # Process and display results with enhanced formatting
        found_matches = 0
        for i, row in enumerate(results, 1):
            print(f"\n🎥 VIDEO #{i}")
            print("-" * 60)
            print(f"📂 URI: {row.uri}")
            
            # Extract video metadata from filename if available
            if row.uri:
                filename = row.uri.split('/')[-1]
                if '_' in filename:
                    parts = filename.split('_')
                    if len(parts) >= 4:
                        camera_id = parts[0]
                        timestamp = parts[1]
                        lat_lon = f"{parts[2]}_{parts[3]}"
                        print(f"📹 Camera: {camera_id}")
                        print(f"🕐 Timestamp: {timestamp}")
                        print(f"📍 Location: {lat_lon}")
            
            print("\n🤖 AI Analysis Result:")
            if row.result:
                # Try to parse JSON and format nicely
                try:
                    analysis = row.result
                    if analysis.get('personFound'):
                        found_matches += 1
                        videos_with_persons.append(row.uri)
                        print("🚨 PERSON DETECTED!")
                        print(f"⭐ Confidence: {analysis.get('confidenceScore', 'N/A')}")
                        print(f"📝 Summary: {analysis.get('summaryOfFindings', 'N/A')}")
                    else:
                        print("❌ No person detected")
                        print(f"📝 Reason: {analysis.get('result', 'N/A')}")
                except (Exception):
                    print(row.result if row.result else "No analysis result")
            else:
                print("⚠️ No analysis result returned")
            
            print("-" * 60)
        
        print("\n📈 ANALYSIS SUMMARY:")
        print(f"   🎥 Total videos analyzed: {results.total_rows}")
        print(f"   ✅ Videos with person detected: {found_matches}")
        print(f"   ❌ Videos without detection: {results.total_rows - found_matches}")
        if found_matches > 0:
            print(f"   🚨 RECOMMEND IMMEDIATE INVESTIGATION OF {found_matches} VIDEO(S)!")
    else:
        print("ℹ️ No surveillance videos found for analysis")

except Exception as e:
    print(f"❌ Error executing video analysis: {str(e)}")
    print(f"🔧 Error type: {type(e).__name__}")
    if hasattr(e, "errors") and e.errors:
        for error in e.errors:
            print(f"📋 Error details: {error}")

🎬 Executing AI video analysis across all surveillance footage...
🤖 Using Gemini 2.5 Pro multimodal model for person detection
✅ Video analysis completed successfully!
📊 Total videos processed: 4
🔧 Query job ID: dd1cda7f-dba7-423d-82f3-c07c473bb940

🎬 SURVEILLANCE VIDEO ANALYSIS RESULTS

🎥 VIDEO #1
------------------------------------------------------------
📂 URI: gs://homeward_videos_7fde5434/CAM004_20240815170000_37.7849_-122.4094_VIRAT_OPENDATA_1920x1080.mp4
📹 Camera: CAM004
🕐 Timestamp: 20240815170000
📍 Location: 37.7849_-122.4094

🤖 AI Analysis Result:
❌ No person detected
📝 Reason: N/A
------------------------------------------------------------

🎥 VIDEO #2
------------------------------------------------------------
📂 URI: gs://homeward_videos_7fde5434/CAM001_20240815123000_37.7849_-122.4094_VIRAT_OPENDATA_1920x1080.mp4
📹 Camera: CAM001
🕐 Timestamp: 20240815123000
📍 Location: 37.7849_-122.4094

🤖 AI Analysis Result:
❌ No person detected
📝 Reason: N/A
----------------------------

In [ ]:
# Extract video URIs with personFound from previous analysis results

print("🔍 EXTRACTING VIDEOS WITH PERSON DETECTIONS")
print("=" * 50)

print(f"\n📊 Found {len(videos_with_persons)} videos with person detections")

# Download and preview only videos with person detections
if len(videos_with_persons) > 0:
    temp_folder = tempfile.mkdtemp()

    print(f"\n🎬 DOWNLOADING AND PREVIEWING {len(videos_with_persons)} VIDEOS WITH DETECTIONS")
    print("=" * 60)
    
    for i, video_uri in enumerate(videos_with_persons, 1):
        print(f"\n🎥 Video #{i} with Person Detection:")
        print(f"📂 URI: {video_uri}")
        
        # Extract filename from URI
        filename = video_uri.split('/')[-1]
        local_path = f"{temp_folder}/{filename}"
        
        # Download video using gsutil
        print("📥 Downloading...")
        download_cmd = f"gsutil cp '{video_uri}' '{local_path}'"
        
        result = os.system(download_cmd)
        if result == 0:
            print("✅ Download successful")
            
            # Display video in notebook
            print("🎬 Video Preview:")
            display(Video(local_path, width=800, height=450))
            
        else:
            print("❌ Download failed")
            
else:
    print("\n📹 No videos with person detections found.")
    print("💡 Note: When videos with personFound=True are available, they will be automatically downloaded and previewed here.")


---
# 🧹 Environment Cleanup

This section provides cleanup functionality to remove all GCP resources created during this demo. This is equivalent to running the `destroy.sh` script but adapted for the notebook environment.

⚠️ **WARNING**: This will permanently delete all resources including:
- Storage bucket and all uploaded videos
- BigQuery dataset and all tables
- BigQuery connection and associated service accounts

**Uncomment and run the cells below only when you want to completely clean up the environment.**

In [137]:
# # DELETE BIGQUERY DATASET
# print("🗑️ Deleting BigQuery dataset...")
# if run_command(f"bq rm -r -f --project_id={PROJECT_ID} {DATASET_ID}"):
#     print(f"✅ BigQuery dataset deleted: {DATASET_ID}")
# else :
#     print("⚠️ Dataset may not exist or already deleted")

# print("💡 Cleanup commands are commented out for safety. Uncomment to execute.")

🗑️ Deleting BigQuery dataset...
✅ Success: bq rm -r -f --project_id={PROJECT_ID} {DATASET_ID}
✅ BigQuery dataset deleted: {DATASET_ID}
💡 Cleanup commands are commented out for safety. Uncomment to execute.


In [ ]:
# DELETE BIGQUERY CONNECTION AND IAM BINDINGS
# print("🗑️ Deleting BigQuery connection...")
# try:
#     connection_full_id = f"{PROJECT_ID}.{LOCATION}.{CONNECTION_ID}"
    
#     # Delete the connection
#     run_command(f"bq rm --connection --location={LOCATION} --project_id={PROJECT_ID} {CONNECTION_ID}")
#     print(f"✅ BigQuery connection deleted: {CONNECTION_ID}")
# except Exception as e:
#     print(f"⚠️ Connection may not exist or already deleted: {e}")

print("💡 Connection cleanup commands are commented out for safety. Uncomment to execute.")

🗑️ Deleting BigQuery connection...
✅ Success: bq rm --connection --location={LOCATION} --project_id={PROJECT_ID} {CONNECTION_ID}
✅ BigQuery connection deleted: {CONNECTION_ID}
💡 Connection cleanup commands are commented out for safety. Uncomment to execute.


In [ ]:
# DELETE STORAGE BUCKET AND ALL CONTENTS
# print("🗑️ Deleting storage bucket and all contents...")
# try:
#     # Check if bucket exists
#     bucket_exists = run_command(f"gsutil ls -b gs://{BUCKET_NAME}")
#     if bucket_exists is not None:  
#         run_command(f"gsutil -m rm gs://{BUCKET_NAME}/**")
#         # Delete the bucket itself
#         run_command(f"gsutil rb gs://{BUCKET_NAME}")
#         print(f"✅ Storage bucket deleted: gs://{BUCKET_NAME}")
#     else:
#         print(f"⚠️ Bucket does not exist: gs://{BUCKET_NAME}")
# except Exception as e:
#     print(f"⚠️ Bucket may not exist or already deleted: {e}")

print("💡 Storage cleanup commands are commented out for safety. Uncomment to execute.")

🗑️ Deleting storage bucket and all contents...
✅ Success: gsutil ls -b gs://{DATASET_ID}_videos_c175266e
   Output: gs://{DATASET_ID}_videos_c175266e/
✅ Success: gsutil -m rm gs://{DATASET_ID}_videos_c175266e/**
✅ Success: gsutil rb gs://{DATASET_ID}_videos_c175266e
✅ Storage bucket deleted: gs://{DATASET_ID}_videos_c175266e
💡 Storage cleanup commands are commented out for safety. Uncomment to execute.
